In [22]:
import numpy as np

import matplotlib as mpl

import matplotlib.pyplot as plt
plt.ion()

# for the symbolic manipulation of jacobian
import sympy as sp
# from sympy import symbols
# from sympy import sin, cos, asin, acos, pi, atan2, sqrt
from sympy.utilities.lambdify import lambdify
# from sympy import Matrix

from scipy.optimize import minimize
from scipy.optimize import fsolve

In [23]:
## Motor constants
K_T = 0.0285;                 # Nm / A
peak_amp = 30;              # A
peak_torque = K_T * peak_amp; # Nm
m_motor = 0.2;              # kg
m_assembly = 0.2;           # kg

m_total = 2*m_motor + m_assembly; # kg

gravity = 9.8;              # m/s^2

weight = m_total * gravity  # N

l1 = 0.09;                  # m 
l2 = 0.16;                  # m
w = 0.07;                   # m


## preparation before active motors

In [34]:
import time
import odrive
import odrive.utils
import odrive.enums

### Connect to ODrive

In [35]:
odrv1 = odrive.find_any()
if odrv1 is not None:
    print('Connected!')
    print('Odrive serial {}'.format(odrv1.serial_number))
    
    m0 = odrv1.axis0.motor.is_calibrated
    m1 = odrv1.axis1.motor.is_calibrated
    
    print('Motor 0 calibrated: {}'.format(m0))
    print('Motor 1 calibrated: {}'.format(m1))    
    
else:
    print('Not connected')

Connected!
Odrive serial 53207020417589
Motor 0 calibrated: False
Motor 1 calibrated: False


### Calibrate

In [19]:
odrv1.axis0.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE
odrv1.axis1.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE

time.sleep(15)

print('\t Odrv 1_Motor 0 calibration result: {} \r\n'.format(odrv1.axis0.motor.is_calibrated), 
      '\t Odrv 1_Motor 1 calibration result: {}'.format(odrv1.axis1.motor.is_calibrated))
odrv1.axis0.encoder.config.use_index = False
odrv1.axis0.encoder.config.pre_calibrated = False
odrv1.axis0.config.startup_encoder_index_search = False
odrv1.axis0.motor.config.pre_calibrated = False
odrv1.save_configuration()

	 Odrv 1_Motor 0 calibration result: True 
 	 Odrv 1_Motor 1 calibration result: True


In [30]:
odrv1.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
odrv1.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

In [24]:
# get motor parameters
motor_cpr = (odrv1.axis0.encoder.config.cpr, odrv1.axis1.encoder.config.cpr)
pos_zero = ([odrv1.axis0.encoder.pos_estimate, odrv1.axis1.encoder.pos_estimate])

# position to angle
angle = np.zeros(shape=(1,2))
def pos2angle(pos, cpr = motor_cpr, pos_zero = pos_zero, zero_theta = (0,0)):
    angle[0,0] = -2*np.pi*(pos[0] - pos_zero[0])/motor_cpr[0]+zero_theta[0]
    angle[0,1] = -2*np.pi*(pos[1] - pos_zero[1])/motor_cpr[1]+zero_theta[1]
    return (angle)

In [25]:
# angle to position
pos = np.zeros(shape=(1,2))
def angle2pos(thetaL, thetaR, pos_zero = pos_zero, cpr = motor_cpr, zero_theta = (0,0)):
    pos[0,0] = -(thetaL - zero_theta[0])/(2*np.pi)*motor_cpr[0] + pos_zero[0]
    pos[0,1] = -(thetaR - zero_theta[1])/(2*np.pi)*motor_cpr[1] + pos_zero[1]    
    return (pos)

In [31]:
odrv1.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
odrv1.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

In [32]:
start = time.time()
t_curr = 0
C = 0.5

pos0 = ([odrv1.axis0.encoder.pos_estimate, odrv1.axis1.encoder.pos_estimate])
aL0 = pos2angle(pos0)[0,0]
aR0 = pos2angle(pos0)[0,1]
    
while(t_curr < 20):
    t_curr = (time.time() - start)
    pos0 = ([odrv1.axis0.encoder.pos_estimate, odrv1.axis1.encoder.pos_estimate])
    aL = pos2angle(pos0)[0,0]
    aR = pos2angle(pos0)[0,1]
    
    aL0 = aL0 + 0.01*(3+0*np.sin(aR-aL))
    aR0 = aR0 + 0.01*(3+50*np.sin(aL-aR))
 
    odrv1.axis0.controller.pos_setpoint = angle2pos(aL0,aR0)[0,0]
    odrv1.axis1.controller.pos_setpoint = angle2pos(aL0,aR0)[0,1]
    
    time.sleep(0.01)

In [33]:
odrv1.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
odrv1.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

In [41]:
pos_zero = ([odrv1.axis0.encoder.pos_estimate,odrv1.axis1.encoder.pos_estimate])
pos_zero

[-1998.765625, 2921.234375]